In [0]:
!pip install wikipedia
!pip install wikipedia-api

In [0]:
import doc_utils
from wiki_parser import concurrentGetAllCatArticles, concurrentGetWikiFullPage
from arxiv_parser import arxiv_parser
from max_sim_classifier import MaxSimClassifier

In [0]:
import random
import numpy as np 
import warnings

warnings.filterwarnings("ignore")

np.random.seed(2020)
random.seed(2020)

In [4]:
arxiv_wiki_topics = ["Computer science",
                    "Economics",
                    "Systems engineering",
                    "Mathematics",
                    "Astrophysics",
                    "Computational biology",
                    "Quantitative finance",
                    "Statistics"]

topic_defs = concurrentGetWikiFullPage(topics_list = arxiv_wiki_topics )

Obtaining full wikipedia page for the topic: Computer science. (Definition of Class #[0])
Obtaining full wikipedia page for the topic: Economics. (Definition of Class #[1])Obtaining full wikipedia page for the topic: Systems engineering. (Definition of Class #[2])

Obtaining full wikipedia page for the topic: Mathematics. (Definition of Class #[3])
Obtaining full wikipedia page for the topic: Astrophysics. (Definition of Class #[4])Obtaining full wikipedia page for the topic: Computational biology. (Definition of Class #[5])
Obtaining full wikipedia page for the topic: Quantitative finance. (Definition of Class #[6])

Obtaining full wikipedia page for the topic: Statistics. (Definition of Class #[7])


In [14]:
dataset, paperslist = arxiv_parser(100)

Retrieved 800 papers in total from 8 subjects (100 from each).


In [0]:
x_train, y_train, x_test, y_test = doc_utils.processClassifierData(topic_defs, 
                                                                   dataset, 
                                                                   arxiv_wiki_topics, 
                                                                   dataset_type="arxiv")
max_sim_model = MaxSimClassifier(dataset_type="arxiv")

In [16]:
max_sim_model.fit(x_train, y_train)

MaxSimClassifier(dataset_type='arxiv', epochs=50, min_count=2, vector_size=50)

In [17]:
max_sim_model.score(dataset,y_test, eval="top2")

0.51125

## Label propagation with "extended" train data

In [19]:
new_dataset, new_x_train, new_y_test = max_sim_model.label_prop(x_train, dataset , paperslist,
                                                                result="extended" ,debug = True )

Topic 0 (computer_science) best matching paper: id #88
	--->True label:[ 0 ]( computer_science ) 		Paper title: KI, Philosophie, Logik
Topic 1 (economics) best matching paper: id #136
	--->True label:[ 1 ]( economics ) 		Paper title: The transmission of liquidity shocks via China's segmented money market: evidence from recent market events
Topic 2 (eess) best matching paper: id #250
	--->True label:[ 2 ]( eess ) 		Paper title: Performance and Scaling of Collaborative Sensing and Networking for Automated Driving Applications
Topic 3 (mathematics) best matching paper: id #384
	--->True label:[ 3 ]( mathematics ) 		Paper title: Coordinates on the augmented moduli space of convex RP^2 structures
Topic 4 (physics) best matching paper: id #480
	--->True label:[ 4 ]( physics ) 		Paper title: Contribution of Cosmic Rays from Sources with a Monoenergetic Proton Spectrum to the Extragalactic Diffuse Gamma-Ray Emission
Topic 5 (q_biology) best matching paper: id #527
	--->True label:[ 5 ]( q_biol

In [20]:
label_prop_model = MaxSimClassifier("arxiv")
label_prop_model.fit(new_x_train, y_train)

MaxSimClassifier(dataset_type='arxiv', epochs=50, min_count=2, vector_size=50)

In [21]:
label_prop_model.score(new_dataset, new_y_test, eval="top2")

0.5416666666666666

## Label propagation with Best papers as train data

In [22]:
new_dataset, new_x_train, new_y_test = max_sim_model.label_prop(x_train, dataset , paperslist,
                                                                result="bestpapers" ,debug = False )

In [23]:
label_prop_model_2 = MaxSimClassifier("arxiv")
label_prop_model_2.fit(new_x_train, y_train)

MaxSimClassifier(dataset_type='arxiv', epochs=50, min_count=2, vector_size=50)

In [24]:
label_prop_model_2.score(new_dataset, new_y_test, eval="top2")

0.33035714285714285